# 날씨 데이터 수집

In [1]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

In [ ]:
# Selenium 웹 드라이버 서비스 설정, 실행
service = Service(executable_path='chromedriver.exe')
driver = webdriver.Chrome(service=service)

# 네이버 메인 페이지로 이동
driver.get("https://www.naver.com")

# 검색 입력 상자 찾기
search_input = driver.find_element(By.ID, "query")  # ID, 클래스 또는 다른 선택자로 요소 찾기

# 검색어 입력
search_input.send_keys("잠실야구장 날씨")

# 검색 실행 (예: 엔터 키를 누르는 것으로 검색을 수행)
search_input.send_keys(Keys.ENTER)

# 페이지가 완전히 로드될 때까지 잠시 대기
time.sleep(5)

In [ ]:
# 날씨 정보 추출
weather_main = driver.find_element(By.CSS_SELECTOR, ".weather_main .blind").text
current_temp = driver.find_element(By.CSS_SELECTOR, ".temperature_text strong").text.replace("현재 온도", "").strip()   

print(f"오늘의 날씨: {weather_main}")
print(f"현재 온도: {current_temp}")

In [ ]:
# 주간 날씨 데이터를 포함하는 요소 찾기
week_items = driver.find_elements(By.CLASS_NAME, "week_item")

# 각 요소에서 날짜, 날씨 상태, 최저기온 및 최고기온 추출
for item in week_items:
    day = item.find_element(By.CSS_SELECTOR, ".cell_date .day").text
    date = item.find_element(By.CSS_SELECTOR, ".cell_date .date").text
    weather_am = item.find_elements(By.CSS_SELECTOR, ".weather_inner")[0].text
    weather_pm = item.find_elements(By.CSS_SELECTOR, ".weather_inner")[1].text
    low_temp = item.find_element(By.CSS_SELECTOR, ".lowest").text
    high_temp = item.find_element(By.CSS_SELECTOR, ".highest").text
    
    print(f"{day} {date} - 오전시간: {weather_am}, 오후시간: {weather_pm}, 기온: {low_temp}/{high_temp}")

In [ ]:
# 오전 데이터 추출
weather_am = item.find_elements(By.CSS_SELECTOR, ".weather_inner")[0]
am_rainfall = weather_am.find_element(By.CSS_SELECTOR, "span.rainfall").text
am_description = weather_am.find_element(By.CSS_SELECTOR, "i.wt_icon span.blind").text

# 오후 데이터 추출
weather_pm = item.find_elements(By.CSS_SELECTOR, ".weather_inner")[1]
pm_rainfall = weather_pm.find_element(By.CSS_SELECTOR, "span.rainfall").text
pm_description = weather_pm.find_element(By.CSS_SELECTOR, "i.wt_icon span.blind").text

print(f"오전 강수확률: {am_rainfall}, 오전 날씨상태: {am_description}")
print(f"오전 강수확률: {pm_rainfall}, 오전 날씨상태: {pm_description}")

In [ ]:
# 종료
driver.quit()

In [2]:
# Selenium 웹 드라이버 서비스 설정, 실행
service = Service(executable_path='chromedriver.exe')
driver = webdriver.Chrome(service=service)

# 지역 목록 정의
locations = ["잠실야구장", "고척스카이돔", "수원KT위즈파크", "인천문학경기장", "대구삼성라이온즈파크", "한화생명이글스파크", "부산사직종합운동장사직야구장", "창원NC파크", "광주기아챔피언스필드"]

# DataFrame을 위한 빈 리스트
data = []

for location in locations:
    driver.get("https://www.naver.com")
    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "query")))
    search_input = driver.find_element(By.ID, "query")
    search_input.send_keys(f"{location} 날씨")
    search_input.send_keys(Keys.ENTER)
    time.sleep(2)
    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "week_item")))
    week_items = driver.find_elements(By.CLASS_NAME, "week_item")

    for item in week_items:
        day = item.find_element(By.CSS_SELECTOR, ".cell_date .day").text
        date = item.find_element(By.CSS_SELECTOR, ".cell_date .date").text

        weather_am = item.find_elements(By.CSS_SELECTOR, ".weather_inner")[0]
        am_rainfall = weather_am.find_element(By.CSS_SELECTOR, "span.rainfall").text
        am_description = weather_am.find_element(By.CSS_SELECTOR, "i.wt_icon span.blind").text
         
        weather_pm = item.find_elements(By.CSS_SELECTOR, ".weather_inner")[1]
        pm_rainfall = weather_pm.find_element(By.CSS_SELECTOR, "span.rainfall").text
        pm_description = weather_pm.find_element(By.CSS_SELECTOR, "i.wt_icon span.blind").text
        
        low_temp = item.find_element(By.CSS_SELECTOR, ".lowest").text.replace("최저기온\n", "").strip() 
        high_temp = item.find_element(By.CSS_SELECTOR, ".highest").text.replace("최고기온\n", "").strip()
        
        data.append({
            '지역': location,
            '날짜': date,
            '요일': day,
            '오전 강수확률': am_rainfall,
            '오전 날씨상태': am_description,
            '오후 강수확률': pm_rainfall,
            '오후 날씨상태': pm_description,
            '최저기온': low_temp,
            '최고기온': high_temp
        })
        
# 종료
driver.quit()

In [3]:
# 데이터 프레임 생성
df = pd.DataFrame(data)
df

,지역,날짜,요일,오전 강수확률,오전 날씨상태,오후 강수확률,오후 날씨상태,최저기온,최고기온
0,잠실야구장,5.13.,오늘,0%,맑음,0%,맑음,12°,24°
1,잠실야구장,5.14.,내일,0%,맑음,0%,맑음,13°,25°
2,잠실야구장,5.15.,수,20%,구름많음,60%,구름많고 가끔 비,14°,19°
3,잠실야구장,5.16.,목,30%,구름많음,0%,맑음,10°,20°
4,잠실야구장,5.17.,금,0%,맑음,0%,맑음,13°,24°
...,...,...,...,...,...,...,...,...,...
85,광주기아챔피언스필드,5.18.,토,0%,맑음,0%,맑음,14°,27°
86,광주기아챔피언스필드,5.19.,일,0%,맑음,0%,맑음,16°,27°
87,광주기아챔피언스필드,5.20.,월,0%,맑음,30%,구름많음,16°,28°
88,광주기아챔피언스필드,5.21.,화,30%,구름많음,30%,구름많음,17°,27°


In [4]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

def collect_weather_data(locations):
    # 웹 드라이버 설정 및 실행
    service = Service(executable_path='chromedriver.exe')
    driver = webdriver.Chrome(service=service)
    
    # 최종 데이터를 저장할 리스트
    data = []

    for location in locations:
        driver.get("https://www.naver.com")
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "query")))
        search_input = driver.find_element(By.ID, "query")
        search_input.send_keys(f"{location} 날씨")
        search_input.send_keys(Keys.ENTER)
        time.sleep(2)

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "week_item")))
        week_items = driver.find_elements(By.CLASS_NAME, "week_item")

        for item in week_items:
            day = item.find_element(By.CSS_SELECTOR, ".cell_date .day").text
            date = item.find_element(By.CSS_SELECTOR, ".cell_date .date").text

            weather_am = item.find_elements(By.CSS_SELECTOR, ".weather_inner")[0]
            am_rainfall = weather_am.find_element(By.CSS_SELECTOR, "span.rainfall").text
            am_description = weather_am.find_element(By.CSS_SELECTOR, "i.wt_icon span.blind").text
            
            weather_pm = item.find_elements(By.CSS_SELECTOR, ".weather_inner")[1]
            pm_rainfall = weather_pm.find_element(By.CSS_SELECTOR, "span.rainfall").text
            pm_description = weather_pm.find_element(By.CSS_SELECTOR, "i.wt_icon span.blind").text
            
            low_temp = item.find_element(By.CSS_SELECTOR, ".lowest").text.replace("최저기온\n", "").strip()
            high_temp = item.find_element(By.CSS_SELECTOR, ".highest").text.replace("최고기온\n", "").strip()
            
            weather_info = {
            '지역': location,
            '날짜': date,
            '요일': day,
            '오전 강수확률': am_rainfall,
            '오전 날씨상태': am_description,
            '오후 강수확률': pm_rainfall,
            '오후 날씨상태': pm_description,
            '최저기온': low_temp,
            '최고기온': high_temp
        }
        weather_data.append(weather_info)
    
    # 웹 드라이버 종료
    driver.quit()

    # 데이터 프레임 생성 및 반환
    return pd.DataFrame(data)

locations = ["잠실야구장", "고척스카이돔", "수원KT위즈파크", "인천문학경기장", "대구삼성라이온즈파크", "한화생명이글스파크", "부산사직종합운동장사직야구장", "창원NC파크", "광주기아챔피언스필드"]
weather_df = collect_weather_data(locations)
print(weather_df)

            지역     날짜  요일 오전 강수확률 오전 날씨상태 오후 강수확률    오후 날씨상태 최저기온 최고기온
0        잠실야구장  5.13.  오늘      0%      맑음      0%         맑음  12°  24°
1        잠실야구장  5.14.  내일      0%      맑음      0%         맑음  13°  25°
2        잠실야구장  5.15.   수     20%    구름많음     60%  구름많고 가끔 비  14°  19°
3        잠실야구장  5.16.   목     30%    구름많음      0%         맑음  10°  20°
4        잠실야구장  5.17.   금      0%      맑음      0%         맑음  13°  24°
..         ...    ...  ..     ...     ...     ...        ...  ...  ...
85  광주기아챔피언스필드  5.18.   토      0%      맑음      0%         맑음  14°  27°
86  광주기아챔피언스필드  5.19.   일      0%      맑음      0%         맑음  16°  27°
87  광주기아챔피언스필드  5.20.   월      0%      맑음     30%       구름많음  16°  28°
88  광주기아챔피언스필드  5.21.   화     30%    구름많음     30%       구름많음  17°  27°
89  광주기아챔피언스필드  5.22.   수     30%    구름많음     30%       구름많음  16°  27°

[90 rows x 9 columns]


In [6]:
df = pd.DataFrame(weather_df)
df

,지역,날짜,요일,오전 강수확률,오전 날씨상태,오후 강수확률,오후 날씨상태,최저기온,최고기온
0,잠실야구장,5.13.,오늘,0%,맑음,0%,맑음,12°,24°
1,잠실야구장,5.14.,내일,0%,맑음,0%,맑음,13°,25°
2,잠실야구장,5.15.,수,20%,구름많음,60%,구름많고 가끔 비,14°,19°
3,잠실야구장,5.16.,목,30%,구름많음,0%,맑음,10°,20°
4,잠실야구장,5.17.,금,0%,맑음,0%,맑음,13°,24°
...,...,...,...,...,...,...,...,...,...
85,광주기아챔피언스필드,5.18.,토,0%,맑음,0%,맑음,14°,27°
86,광주기아챔피언스필드,5.19.,일,0%,맑음,0%,맑음,16°,27°
87,광주기아챔피언스필드,5.20.,월,0%,맑음,30%,구름많음,16°,28°
88,광주기아챔피언스필드,5.21.,화,30%,구름많음,30%,구름많음,17°,27°


In [7]:
# 요일 데이터 변경
df.loc[df['요일'] == '오늘', '요일'] = '월'
df.loc[df['요일'] == '내일', '요일'] = '화'

In [8]:
# '날짜' 열의 마지막 점 제거
df['날짜'] = df['날짜'].str.rstrip('.')

In [11]:
print(df['날짜'])

0     5.13
1     5.14
2     5.15
3     5.16
4     5.17
      ... 
85    5.18
86    5.19
87    5.20
88    5.21
89    5.22
Name: 날짜, Length: 90, dtype: object


In [14]:
# 문자열을 datetime 객체로 변환, 연도 정보 추가
df['날짜'] = pd.to_datetime('2024-' + df['날짜'].str.replace('.', '-'))
# datetime 포맷을 월.일로 변경
df['날짜'] = df['날짜'].dt.strftime('%m.%d')

In [15]:
df

,지역,날짜,요일,오전 강수확률,오전 날씨상태,오후 강수확률,오후 날씨상태,최저기온,최고기온
0,잠실야구장,05.13,월,0%,맑음,0%,맑음,12°,24°
1,잠실야구장,05.14,화,0%,맑음,0%,맑음,13°,25°
2,잠실야구장,05.15,수,20%,구름많음,60%,구름많고 가끔 비,14°,19°
3,잠실야구장,05.16,목,30%,구름많음,0%,맑음,10°,20°
4,잠실야구장,05.17,금,0%,맑음,0%,맑음,13°,24°
...,...,...,...,...,...,...,...,...,...
85,광주기아챔피언스필드,05.18,토,0%,맑음,0%,맑음,14°,27°
86,광주기아챔피언스필드,05.19,일,0%,맑음,0%,맑음,16°,27°
87,광주기아챔피언스필드,05.20,월,0%,맑음,30%,구름많음,16°,28°
88,광주기아챔피언스필드,05.21,화,30%,구름많음,30%,구름많음,17°,27°


In [16]:
# 지역 데이터 기반으로 컬럼 추가
# (예)'지역' 컬럼이 '잠실야구장'인 경우 'city' 컬럼에 'Jamsil'을 저장
# '지역' 컬럼을 기반으로 'city' 컬럼을 설정
def map_city(region):
    if region == '잠실야구장':
        return 'Jamsil'
    elif region == '고척스카이돔':
        return 'Gocheok'
    elif region == '수원KT위즈파크':
        return 'Suwon'
    elif region == '인천문학경기장':
        return 'Incheon'
    elif region == '대구삼성라이온즈파크':
        return 'Daegu'
    elif region == '한화생명이글스파크':
        return 'Daejeon'
    elif region == '부산사직종합운동장사직야구장':
        return 'Busan'
    elif region == '창원NC파크':
        return 'Changwon'
    elif region == '광주기아챔피언스필드':
        return 'Gwangju'
    else:
        return None  # 지역이 목록에 없는 경우 None 반환

# 'city' 컬럼 생성
df['city'] = df['지역'].apply(map_city)

# 결과 출력
df

,지역,날짜,요일,오전 강수확률,오전 날씨상태,오후 강수확률,오후 날씨상태,최저기온,최고기온,city
0,잠실야구장,05.13,월,0%,맑음,0%,맑음,12°,24°,Jamsil
1,잠실야구장,05.14,화,0%,맑음,0%,맑음,13°,25°,Jamsil
2,잠실야구장,05.15,수,20%,구름많음,60%,구름많고 가끔 비,14°,19°,Jamsil
3,잠실야구장,05.16,목,30%,구름많음,0%,맑음,10°,20°,Jamsil
4,잠실야구장,05.17,금,0%,맑음,0%,맑음,13°,24°,Jamsil
...,...,...,...,...,...,...,...,...,...,...
85,광주기아챔피언스필드,05.18,토,0%,맑음,0%,맑음,14°,27°,Gwangju
86,광주기아챔피언스필드,05.19,일,0%,맑음,0%,맑음,16°,27°,Gwangju
87,광주기아챔피언스필드,05.20,월,0%,맑음,30%,구름많음,16°,28°,Gwangju
88,광주기아챔피언스필드,05.21,화,30%,구름많음,30%,구름많음,17°,27°,Gwangju


In [17]:
# CSV 파일로 저장
df.to_csv('weather_data.csv', index=False, encoding='utf-8-sig')

In [12]:
# CSV 파일 읽기
file_path = r'D:\ICTEDU\Bigdata\Pandas\Project_Crawling\example_data.csv'
df = pd.read_csv(file_path)

In [13]:
# JSON 파일로 저장
df.to_json('weather_data.json', orient='records', force_ascii=False, lines=True)

In [ ]:
# 날짜 데이터 전처리하려면 아래 코드로!
from datetime import datetime
# 현재 연도 가져오기
current_year = datetime.now().year

# 날짜 변환 함수
def convert_date(date_str):
    # 현재 연도를 추가하여 날짜 파싱
    date_str = f"{current_year}-{date_str.replace('.', '-')}"
    # 불필요한 마지막 '-' 제거 후 날짜 객체로 변환
    date_obj = datetime.strptime(date_str[:-1], '%Y-%m-%d')
    return date_obj

# '날짜' 컬럼 변환
df['날짜'] = df['날짜'].apply(convert_date)
print(df)

In [ ]:
df

In [ ]:
# 데이터 프레임을 JSON으로 저장
df.to_json('weather_data.json', orient='records', force_ascii=False,  date_format='iso')

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def collect_stock_data():
    # Selenium 웹 드라이버 서비스 설정, 실행
    chrome_driver_path = '/home/ict/PycharmProject/teamProject/teamproject/chromedriver'
    service = Service(chrome_driver_path)
    driver = webdriver.Chrome(service=service)

    # 네이버 메인 페이지로 이동
    driver.get("https://www.naver.com")

    # 증권 탭의 링크 주소 얻기
    finance_link = driver.find_element(By.XPATH, '//*[@id="shortcutArea"]/ul/li[6]/a')
    finance_url = finance_link.get_attribute('href')

    # 링크 URL로 직접 이동
    driver.get(finance_url)
    time.sleep(2)

    # 검색 페이지 URL 저장
    search_page_url = driver.current_url

    # 검색할 종목 코드 설정
    search_codes = {'000270': '기아',
                    '030200': 'KT',
                    '003550': 'LG',
                    '036570': '엔씨소프트',
                    '004170': '신세계',
                    '000150': '두산',
                    '004990': '롯데지주',
                    '005930': '삼성전자',
                    '039490': '키움증권',
                    '000880': '한화'}

    all_stock_data = []  # 모든 종목의 데이터를 저장할 리스트

    for search_code in search_codes:
        # 검색 페이지로 다시 이동
        driver.get(search_page_url)
        time.sleep(2)

        # 검색 필드가 로드될 때까지 기다리기
        search_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "stock_items"))
        )

        # 검색 필드 초기화
        search_field.clear()

        # JavaScript를 사용하여 검색 필드에 값 설정
        driver.execute_script("arguments[0].value = arguments[1];", search_field, search_code)

        # ENTER 키를 보내 검색 실행
        search_field.send_keys(Keys.ENTER)
        time.sleep(5)

        # 종목의 시세 페이지로 이동
        market_price_link = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/ul/li[2]/a'))
        )
        market_price_link.click()
        time.sleep(2)

        # 일별 시세 테이블이 iframe에 있음
        iframe_url = f"{finance_url}/item/sise_day.naver?code={search_code}"

        # iframe URL로 직접 이동
        driver.get(iframe_url)

        # 데이터를 저장할 빈 리스트 생성
        stock_data = []

        for page_num in range(1, 13):
            table = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'table.type2')))
            trs = table.find_elements(By.CSS_SELECTOR, 'tbody > tr')
            paginate = driver.find_element(By.XPATH, '/html/body/table[2]/tbody/tr/td')

            # 현재 몇 페이지 크롤링 중인지, 출력
            print("현재 페이지: ", page_num)

            # 테이블에서 모든 행을 순회
            for tr in trs:
                spans = tr.find_elements(By.TAG_NAME, 'span')

                # 각 행에서 기대하는 최소한의 span 수 확인
                if len(spans) < 8:
                    continue  # span 요소가 8개 미만이면 다음 행으로 넘어감

                # 야구 정규 시즌 3월 부터 시작
                target_date = '2024.02.29'
                date_text = spans[0].text
                # print("읽은 날짜:", date_text)  # 디버깅을 위한 출력

                if date_text == target_date:
                    print("목표 날짜에 도달했습니다. 데이터 수집을 종료합니다.")
                    break  # 내부 반복문 종료

                # 유효한 행인 경우, 데이터 추출
                stock_info = {
                    '종목명': search_codes[search_code],  # 종목명을 딕셔너리에서 가져와 저장
                    '날짜': spans[0].text,
                    '종가': spans[1].text,
                    '전일비상태': spans[2].text,
                    '전일비': spans[3].text,
                    '시가': spans[4].text,
                    '고가': spans[5].text,
                    '저가': spans[6].text,
                    '거래량': spans[7].text
                }
                stock_data.append(stock_info)

            if date_text == target_date:
                # print("외부 반복문을 종료합니다.")
                break

            # 다음 페이지로 이동
            try:
                # 페이지가 1~9, 11~19, 21~29
                if page_num % 10 != 0:
                    next_link = driver.find_element(By.XPATH, f"//a[contains(text(), '{page_num + 1}')]")
                    next_link.click()
                # 페이지가 10, 20, 30,...
                else:
                    next_link = driver.find_element(By.XPATH, "//a[contains(text(), '다음')]")
                    next_link.click()
                time.sleep(2)
            except:
                print("마지막 페이지에 도달했습니다.")
                break

        all_stock_data.extend(stock_data)  # 각 종목의 데이터를 전체 데이터 리스트에 추가

    # 종료
    driver.quit()

    # 모든 데이터를 DataFrame으로 변환
    df = pd.DataFrame(all_stock_data)
    # 날짜 열을 datetime 객체로 변환
    df['날짜'] = pd.to_datetime(df['날짜'])
    # datetime 포맷을 월.일로 변경
    df['날짜'] = df['날짜'].dt.strftime('%m.%d')
    # CSV 파일로 저장
    file_path= '/home/ict/PycharmProject/teamProject/teamproject/static/json'
    df.to_csv(f'{file_path}/naver_stocks.csv', index=False, encoding='utf-8-sig')

    return all_stock_data

# 함수 호출
collect_stock_data()


def collect_weather_data(locations):
    # Selenium 웹 드라이버 서비스 설정, 실행
    chrome_driver_path = '/home/ict/PycharmProject/teamProject/teamproject/chromedriver'
    service = Service(chrome_driver_path)
    driver = webdriver.Chrome(service=service)

    # 최종 데이터를 저장할 리스트
    weather_data = []

    for location in locations:
        driver.get("https://www.naver.com")
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "query")))
        search_input = driver.find_element(By.ID, "query")
        search_input.send_keys(f"{location} 날씨")
        search_input.send_keys(Keys.ENTER)
        time.sleep(2)

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "week_item")))
        week_items = driver.find_elements(By.CLASS_NAME, "week_item")

        for item in week_items:
            day = item.find_element(By.CSS_SELECTOR, ".cell_date .day").text
            date = item.find_element(By.CSS_SELECTOR, ".cell_date .date").text

            weather_am = item.find_elements(By.CSS_SELECTOR, ".weather_inner")[0]
            am_rainfall = weather_am.find_element(By.CSS_SELECTOR, "span.rainfall").text
            am_description = weather_am.find_element(By.CSS_SELECTOR, "i.wt_icon span.blind").text

            weather_pm = item.find_elements(By.CSS_SELECTOR, ".weather_inner")[1]
            pm_rainfall = weather_pm.find_element(By.CSS_SELECTOR, "span.rainfall").text
            pm_description = weather_pm.find_element(By.CSS_SELECTOR, "i.wt_icon span.blind").text

            low_temp = item.find_element(By.CSS_SELECTOR, ".lowest").text.replace("최저기온\n", "").strip()
            high_temp = item.find_element(By.CSS_SELECTOR, ".highest").text.replace("최고기온\n", "").strip()

            weather_info = {
                '지역': location,
                '날짜': date,
                '요일': day,
                '오전 강수확률': am_rainfall,
                '오전 날씨상태': am_description,
                '오후 강수확률': pm_rainfall,
                '오후 날씨상태': pm_description,
                '최저기온': low_temp,
                '최고기온': high_temp
            }
            weather_data.append(weather_info)

    # 웹 드라이버 종료
    driver.quit()

    # 모든 데이터를 DataFrame으로 변환
    df = pd.DataFrame(weather_data)
    # 요일 데이터 변경
    df.loc[df['요일'] == '오늘', '요일'] = '월'
    df.loc[df['요일'] == '내일', '요일'] = '화'
    # '날짜' 열의 마지막 점 제거
    df['날짜'] = df['날짜'].str.rstrip('.')
    # 문자열을 datetime 객체로 변환, 연도 정보 추가
    df['날짜'] = pd.to_datetime('2024-' + df['날짜'].str.replace('.', '-'))
    # datetime 포맷을 월.일로 변경
    df['날짜'] = df['날짜'].dt.strftime('%m.%d')

    # 지역 데이터 기반으로 컬럼 추가
    # (예)'지역' 컬럼이 '잠실야구장'인 경우 'city' 컬럼에 'Jamsil'을 저장
    # '지역' 컬럼을 기반으로 'city' 컬럼을 설정
    def map_city(region):
        if region == '잠실야구장':
            return 'Jamsil'
        elif region == '고척스카이돔':
            return 'Gocheok'
        elif region == '수원KT위즈파크':
            return 'Suwon'
        elif region == '인천문학경기장':
            return 'Incheon'
        elif region == '대구삼성라이온즈파크':
            return 'Daegu'
        elif region == '한화생명이글스파크':
            return 'Daejeon'
        elif region == '부산사직종합운동장사직야구장':
            return 'Busan'
        elif region == '창원NC파크':
            return 'Changwon'
        elif region == '광주기아챔피언스필드':
            return 'Gwangju'
        else:
            return None  # 지역이 목록에 없는 경우 None 반환

    # 'city' 컬럼 생성
    df['city'] = df['지역'].apply(map_city)
    # CSV 파일로 저장
    file_path = '/home/ict/PycharmProject/teamProject/teamproject/static/json'
    df.to_csv(f'{file_path}/weather_info.csv', index=False, encoding='utf-8-sig')

    return weather_data

# 함수 호출
locations = ["잠실야구장", "고척스카이돔", "수원KT위즈파크", "인천문학경기장", "대구삼성라이온즈파크", "한화생명이글스파크", "부산사직종합운동장사직야구장", "창원NC파크",
             "광주기아챔피언스필드"]
collect_weather_data(locations)


In [ ]:
import pandas as pd
from django.http import JsonResponse, HttpResponse

def loadJson1(request):
    # CSV 파일 경로
    pathv1 = '/home/ict/PycharmProject/teamProject/teamproject/static/json/naver_stocks.csv'

    try:
        # CSV 파일을 DataFrame으로 로드
        df = pd.read_csv(pathv1)

        # DataFrame을 JSON으로 변환하여 응답
        resJson = df.to_json('naver_stocks.json', orient='records', force_ascii=False)

        # JSON 응답 반환
        return JsonResponse(resJson, safe=False)

    except Exception as e:
        # 예외가 발생했을 경우 처리
        return JsonResponse({'error': str(e)}, status=500)

def loadJson2(request):
    # CSV 파일 경로
    pathv2 = '/home/ict/PycharmProject/teamProject/teamproject/static/json/weather_info.csv'

    try:
        # CSV 파일을 DataFrame으로 로드
        df = pd.read_csv(pathv2)

        # DataFrame을 JSON으로 변환하여 응답
        resJson = df.to_json('weather_info.json', orient='records', force_ascii=False)

        # JSON 응답 반환
        return JsonResponse(resJson, safe=False)

    except Exception as e:
        # 예외가 발생했을 경우 처리
        return JsonResponse({'error': str(e)}, status=500)